In [ ]:
import requests
import pandas as pd

df = pd.read_csv('https://data.nasa.gov/api/views/gh4g-9sfh/rows.csv?accessType=DOWNLOAD')

df

In [ ]:
##Hemisferios nuevos 
df['Hemisphere_ns'] = df['reclat'].apply(lambda x: 'North' if x >= 0 else 'South')


df['Hemisphere_ew'] = df['reclong'].apply(lambda x: 'East' if x >= 0 else 'West')

df

In [ ]:
df.dropna(inplace=True)
df

In [ ]:
import geopandas as gpd
import pandas as pd
import earthpy as et
import shapely
from folium.plugins import MarkerCluster
from shapely.wkt import loads as wkt_loads 
from shapely.geometry import Point

from shapely.geometry import Point

shapefile_path = '/workspaces/Meteo/data/raw/country_shapes.shp'
gdf_countries = gpd.read_file(shapefile_path)
df['geometry'] = df.apply(lambda row: Point(row['reclong'], row['reclat']), axis=1)
gdf_points = gpd.GeoDataFrame(df, geometry='geometry')
gdf_points = gdf_points.set_crs(gdf_countries.crs)
gdf_merged = gpd.sjoin(gdf_points, gdf_countries, how='left', op='within')
df.reset_index(drop=True, inplace=True)
gdf_merged.reset_index(drop=True, inplace=True)


if len(gdf_merged) == len(df):
    df['cntry_name'] = gdf_merged['cntry_name']
else:
    
    df = df.join(gdf_merged['cntry_name'])


df

In [ ]:
##Pais que mas se repite

most_country = gdf_merged['cntry_name'].value_counts().idxmax()
most_country

In [ ]:
##TOP 10 de paises

country_counts = gdf_merged['cntry_name'].value_counts()

top_10 = country_counts.head(10)

top_10

In [ ]:
##Agrupar peso de meteos en 10(?)
pd.qcut() agrupa segun frecuencia 
df['mass_group'] = pd.qcut(df['mass (g)'], 10, labels=False)


print(df[['mass (g)', 'mass_group']])

In [ ]:
###NO RUN POR QUE SI NO QUEDA LA CAGA
import folium
center_lat = df['reclat'].mean()
center_long = df['reclong'].mean()


m = folium.Map(location=[center_lat, center_long], zoom_start=4)


for index, row in df.iterrows():
    folium.Marker([row['reclat'], row['reclong']]).add_to(m)


m